In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [3]:
def data_concat(df,path):
    for p in path:
        data = pd.read_csv(p,header=None)
        df = pd.concat([df,data],axis=0)
    return df

In [4]:
# 取PSD特征
def psd_feature(df):
    m=df.shape[1]
    n=100   #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        df_n = 10**(df_n/10)
        mean=df_n.mean(axis=1)
        var = df_n.var(axis=1)
        mean=mean.map(lambda x :math.log(x,10)*10)
        signal_feature['psd_mean_'+str(i)] = mean
        signal_feature['psd_var_'+str(i)] = var
    return signal_feature

In [5]:
# 取PSD差分特征
def psd_cha_feature(df):
    m=df.shape[1]
    n=100
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        mean=df_n.mean(axis=1)    # mean 代表该频段的平均上升下降次数
        #mean=mean.map(lambda x:0 if x>-0.5 else 1) # 二次离散化
        signal_feature['psd_cha_'+str(i)] = mean
    return signal_feature

In [6]:
def convert_cha(sa):
    if (sa>0):
        return 1
    else:
        return -1

In [7]:
# 取差值（斜率）作为特征
def cha(df):
    n=df.shape[1]
    ret = pd.DataFrame()
    for i in range(n-1):
        d=df.iloc[:,i+1]-df.iloc[:,i]
        ret[i]=d
    ret[n-1]=0
    return ret

In [1]:
path_all=['./matlab/data_UAV_f_20.csv','./matlab/data_UAV_f_10.csv','./matlab/data_UAV_f_0.csv','./matlab/data_UAV_f_-10.csv','./matlab/data_UAV_f_-12.csv',\
         './matlab/data_UAV_f_-15.csv','./matlab/data_noise_f.csv']

In [8]:
all_data = pd.DataFrame()
all_data = data_concat(all_data,path_all)
all_data.shape

(9000, 5000)

In [9]:
all_data=all_data.iloc[:,range(1500,4500)]
all_data.shape

(9000, 3000)

In [10]:
all_data_cha = cha(all_data)
all_data_cha.shape

(9000, 3000)

In [34]:
all_data_cha = smooth(all_data_cha)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
all_data.index=range(all_data.shape[0])
all_data_cha.index=range(all_data_cha.shape[0])

In [36]:
# train_feature 为最终的训练特征
all_feature = pd.DataFrame()

In [37]:
psd = psd_feature(all_data)
psd = psd.sort_index(axis=1)
#psd_mean_feature=psd.iloc[:,range(30)].applymap(convert_data)
#psd_var_feature=psd.iloc[:,range(30,60)].applymap(convert_var)

In [38]:
all_data_cha_cate = all_data_cha.applymap(convert_cha)
psd_cha = psd_cha_feature(all_data_cha_cate)
psd_cha.shape

(9000, 30)

In [39]:
all_feature = pd.concat([psd,psd_cha],axis=1)
all_feature.shape

(9000, 30)

In [40]:
all_feature

,psd_cha_0,psd_cha_1,psd_cha_2,psd_cha_3,psd_cha_4,psd_cha_5,psd_cha_6,psd_cha_7,psd_cha_8,psd_cha_9,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
0,0.28,-0.24,-0.12,-0.08,0.18,-0.14,0.12,-0.32,-0.20,0.18,...,-0.22,0.00,0.02,0.38,-0.26,-1.00,0.12,-0.52,0.30,-0.38
1,0.22,-0.06,0.06,-0.12,0.24,-0.16,0.10,-0.20,0.16,-0.14,...,-0.16,0.12,-0.16,0.14,-0.16,-1.00,-0.38,0.18,-0.08,0.06
2,0.30,-0.12,0.06,-0.08,0.22,0.06,-0.08,-0.04,0.02,0.10,...,0.04,-0.02,-0.12,0.10,-0.34,-0.98,-0.28,0.04,-0.14,-0.10
3,0.26,-0.02,0.14,-0.14,0.20,-0.10,0.28,-0.14,-0.10,-0.40,...,0.12,-0.02,-0.08,0.06,-0.30,-0.92,-0.56,0.02,0.00,-0.18
4,0.22,0.00,-0.08,0.22,-0.04,-0.12,0.04,0.12,-0.24,0.22,...,-0.08,-0.06,0.10,0.14,-0.28,-1.00,-0.86,0.20,0.08,-0.20
5,0.28,0.04,-0.18,-0.22,0.18,-0.02,0.00,-0.10,-0.24,0.32,...,0.00,0.42,-0.16,0.12,-0.26,-0.92,-0.14,-0.16,-0.06,0.12
6,0.28,-0.02,0.20,-0.08,0.12,-0.26,0.08,0.22,0.18,-0.10,...,-0.08,0.30,-0.08,0.02,-0.24,-0.96,-0.34,0.00,-0.16,0.06
7,0.24,-0.20,0.06,0.36,-0.04,-0.24,0.14,0.00,-0.12,-0.26,...,0.14,0.02,0.16,0.06,-0.24,-0.94,-0.50,-0.02,-0.14,0.08
8,0.22,-0.28,0.14,-0.12,0.08,0.10,0.06,-0.20,-0.10,-0.02,...,-0.08,0.02,-0.08,0.04,-0.32,-1.00,-0.30,-0.08,-0.32,0.54
9,0.30,-0.06,0.18,0.14,-0.30,0.26,-0.22,0.22,-0.20,0.20,...,-0.18,-0.18,0.00,0.16,-0.30,-1.00,-0.16,-0.06,-0.06,-0.18


In [44]:
all_feature.iloc[range(3000,4000),:]

,psd_cha_0,psd_cha_1,psd_cha_2,psd_cha_3,psd_cha_4,psd_cha_5,psd_cha_6,psd_cha_7,psd_cha_8,psd_cha_9,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
3000,0.06,0.16,-0.18,0.08,-0.08,0.08,0.00,-0.32,0.36,-0.18,...,0.02,0.18,-0.24,0.20,-0.14,-0.14,0.30,0.08,-0.20,0.00
3001,0.06,0.14,-0.28,0.26,0.02,-0.28,0.10,-0.04,0.18,0.10,...,-0.22,0.24,-0.18,0.10,-0.10,-0.10,-0.06,0.00,0.30,0.18
3002,-0.10,-0.16,0.08,0.16,-0.02,-0.06,-0.06,0.06,-0.14,0.06,...,0.16,-0.04,-0.04,0.22,0.00,-0.02,-0.02,0.26,-0.02,0.04
3003,0.20,-0.38,0.08,-0.04,0.06,0.14,-0.18,-0.10,0.12,-0.10,...,0.20,-0.18,-0.32,0.02,-0.10,-0.08,0.14,-0.28,0.26,0.14
3004,0.12,0.10,0.00,-0.18,-0.22,0.30,0.00,0.28,-0.12,0.08,...,-0.24,-0.14,-0.04,0.22,-0.08,-0.24,-0.08,0.28,-0.30,-0.14
3005,0.06,-0.16,0.02,0.14,-0.06,-0.30,-0.02,0.02,0.06,-0.14,...,0.02,0.06,-0.12,-0.20,-0.18,0.00,-0.02,0.10,0.02,0.08
3006,-0.18,0.02,-0.04,-0.08,0.18,0.06,0.00,0.40,-0.30,0.10,...,-0.30,0.34,-0.30,0.14,-0.06,0.24,-0.02,0.00,0.06,0.06
3007,-0.18,0.36,-0.42,0.08,-0.12,-0.02,-0.04,0.30,-0.32,0.46,...,0.06,-0.22,0.24,0.12,0.12,-0.42,-0.06,-0.06,0.10,-0.06
3008,0.00,0.04,0.36,-0.12,0.06,0.10,-0.02,-0.02,0.08,0.14,...,0.18,-0.24,0.08,-0.08,0.02,-0.10,-0.38,0.18,-0.20,0.04
3009,0.00,-0.18,-0.16,-0.04,0.22,-0.12,-0.08,-0.06,0.10,0.12,...,0.10,-0.06,0.14,-0.10,-0.10,-0.10,0.12,-0.12,-0.02,0.24


In [16]:
data_all=MinMaxScaler().fit_transform(all_feature) #normalization 得到numpy array

In [17]:
all_data = pd.DataFrame(data_all)

In [18]:
all_data.columns=all_feature.columns

In [19]:
all_data['label']=[1]*6000+[0]*3000
all_data

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29,label
0,0.997791,0.994916,0.991757,0.998812,0.996287,0.994519,0.997878,0.997040,0.997385,0.992860,...,0.454545,0.347826,0.454545,0.166667,0.18750,0.391304,0.318182,0.541667,0.476190,1
1,0.995899,0.993418,0.992894,0.995907,0.997553,0.994307,0.994983,0.996529,0.996737,0.992774,...,0.272727,0.478261,0.136364,0.133333,0.21875,0.434783,0.409091,0.458333,0.476190,1
2,0.994558,0.993525,0.995899,0.994907,0.998328,0.992879,0.991801,0.997180,0.994494,0.993217,...,0.409091,0.260870,0.409091,0.300000,0.34375,0.521739,0.318182,0.375000,0.476190,1
3,0.992625,0.994077,0.995215,0.993277,0.995674,0.993568,0.992411,0.992971,0.994312,0.996180,...,0.454545,0.521739,0.636364,0.300000,0.18750,0.347826,0.318182,0.583333,0.523810,1
4,0.997144,0.993779,0.992374,0.994305,0.997150,0.995314,0.993228,0.996959,0.993863,0.992550,...,0.636364,0.478261,0.409091,0.366667,0.25000,0.173913,0.500000,0.666667,0.285714,1
5,0.993570,0.996410,0.996847,0.994942,0.998540,0.994164,0.997696,0.994984,0.989902,0.991504,...,0.272727,0.478261,0.409091,0.200000,0.40625,0.391304,0.454545,0.541667,0.476190,1
6,0.996533,0.992977,0.992737,0.993570,0.994169,0.994985,0.996736,0.995746,0.995210,0.995179,...,0.409091,0.608696,0.454545,0.400000,0.21875,0.478261,0.636364,0.541667,0.619048,1
7,1.000000,0.995062,0.991975,0.993484,0.996105,0.995175,0.993876,0.997456,0.994172,0.994303,...,0.318182,0.608696,0.454545,0.300000,0.25000,0.347826,0.454545,0.500000,0.476190,1
8,0.997181,0.995935,0.995225,0.996216,0.997437,0.993957,0.994392,0.991673,0.994075,0.995264,...,0.454545,0.347826,0.590909,0.333333,0.28125,0.304348,0.272727,0.458333,0.523810,1
9,0.993912,0.995006,0.991334,0.994788,0.998453,0.992823,0.993992,0.997850,0.992826,0.992432,...,0.181818,0.478261,0.454545,0.266667,0.34375,0.434783,0.500000,0.416667,0.523810,1


In [21]:
all_data.to_csv('train_data_1221.csv',index=False)

In [28]:
all_data.iloc[range(0,1000),:]

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29,label
0,0.997791,0.994916,0.991757,0.998812,0.996287,0.994519,0.997878,0.997040,0.997385,0.992860,...,0.454545,0.347826,0.454545,0.166667,0.18750,0.391304,0.318182,0.541667,0.476190,1
1,0.995899,0.993418,0.992894,0.995907,0.997553,0.994307,0.994983,0.996529,0.996737,0.992774,...,0.272727,0.478261,0.136364,0.133333,0.21875,0.434783,0.409091,0.458333,0.476190,1
2,0.994558,0.993525,0.995899,0.994907,0.998328,0.992879,0.991801,0.997180,0.994494,0.993217,...,0.409091,0.260870,0.409091,0.300000,0.34375,0.521739,0.318182,0.375000,0.476190,1
3,0.992625,0.994077,0.995215,0.993277,0.995674,0.993568,0.992411,0.992971,0.994312,0.996180,...,0.454545,0.521739,0.636364,0.300000,0.18750,0.347826,0.318182,0.583333,0.523810,1
4,0.997144,0.993779,0.992374,0.994305,0.997150,0.995314,0.993228,0.996959,0.993863,0.992550,...,0.636364,0.478261,0.409091,0.366667,0.25000,0.173913,0.500000,0.666667,0.285714,1
5,0.993570,0.996410,0.996847,0.994942,0.998540,0.994164,0.997696,0.994984,0.989902,0.991504,...,0.272727,0.478261,0.409091,0.200000,0.40625,0.391304,0.454545,0.541667,0.476190,1
6,0.996533,0.992977,0.992737,0.993570,0.994169,0.994985,0.996736,0.995746,0.995210,0.995179,...,0.409091,0.608696,0.454545,0.400000,0.21875,0.478261,0.636364,0.541667,0.619048,1
7,1.000000,0.995062,0.991975,0.993484,0.996105,0.995175,0.993876,0.997456,0.994172,0.994303,...,0.318182,0.608696,0.454545,0.300000,0.25000,0.347826,0.454545,0.500000,0.476190,1
8,0.997181,0.995935,0.995225,0.996216,0.997437,0.993957,0.994392,0.991673,0.994075,0.995264,...,0.454545,0.347826,0.590909,0.333333,0.28125,0.304348,0.272727,0.458333,0.523810,1
9,0.993912,0.995006,0.991334,0.994788,0.998453,0.992823,0.993992,0.997850,0.992826,0.992432,...,0.181818,0.478261,0.454545,0.266667,0.34375,0.434783,0.500000,0.416667,0.523810,1


In [31]:
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [32]:
def smooth(df):
    for i in range(len(df)):
        y = df.ix[i].values
        yhat = savitzky_golay(y, 101, 3)
        df.ix[i]=yhat
    return df